In [23]:
## Enable matplotlib inline
%matplotlib inline
import matplotlib.pyplot as plt

## Imports
import pandas as pd
pd.set_option('mode.chained_assignment',None)
pd.set_option('display.mpl_style', 'default') 
pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

import numpy as np

In [24]:
import math

## --------------------------------------------------
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [25]:
## --------------------------------------------------
def trip_work(trip):
    """
    Calculates the score for a trip
    """
    
    work = 0.0
    total_weight = 10.0 + trip['Weight'].sum()
    lon = 0
    lat = 90
    
    for i, row in trip.iterrows():
        current_lon = row['Longitude']
        current_lat = row['Latitude']
        current_w   = row['Weight']
        
        distance = haversine(lon, lat, current_lon, current_lat)
        work += distance * total_weight
        
        total_weight -= current_w
        lon = current_lon
        lat = current_lat
        
    work += haversine(lon, lat, 0, 90) * 10.0
    
    return work
        

In [26]:
## --------------------------------------------------
def make_trips(df, maxd=1.0):
    """
    Make the trips by longitudinal binning and latitude ordering
    """
    
    df = df.sort_values(by='Longitude')
    
    ## Bin the longitude axis such that every bin sums up to 990 pounds
    trip_numbers = []
    trip_number = 1
    weight = 10.0
    longitude = gifts_df['Longitude'].values[0]
    max_delta_longitude = maxd

    for i, row in df.iterrows():
        current_weight = row['Weight']
        current_longitude = row['Longitude']
        if (weight + current_weight > 1000.0) or (current_longitude - longitude > max_delta_longitude):
            trip_number += 1
            weight = 10.0 + current_weight
            longitude = current_longitude
        else:
            weight += current_weight
        trip_numbers.append((row['GiftId'], trip_number))
        
    trip_numbers = np.array(trip_numbers, dtype=int)
    trip_numbers = pd.DataFrame(trip_numbers, columns=['GiftId', 'TripId'])
    
    df = pd.merge(df, trip_numbers)
    
    df = df.sort_values(by=['TripId','Latitude'], ascending=[True,False])
    
    return df
    

In [27]:
## --------------------------------------------------
def score(df):
    """
    Calculates the total score on the entire dataset
    """
    
    n = df['TripId'].max()
    x = 0
    for i in range(1, n+1):
        trip = df[df['TripId'] == i]
        x += trip_work(trip)
    return x, n

In [44]:
## --------------------------------------------------
def partition(df, gap=31, maxd=1.0):
    """
    partition the dataset by ocean gaps
    """
    
    partitions = [[]]
    
    n = df['TripId'].max()

    ## Distribute the gifts across the partitions
    for i in range(1, n+1):
        trip = df[df['TripId'] == i]
    
        trip['gaps'] = -trip['Latitude'].diff()
        n_gaps = len(trip[trip['gaps'] > gap])
        n_partitions = len(partitions)-1
        
        if n_partitions < n_gaps:
            for i in range(n_gaps - n_partitions):
                partitions.append([])
        
        region = 0
    
        for index, row in trip.iterrows():
            g = row['gaps']
            gift = row['GiftId']
            
            if g > gap:
                region += 1
            
            partitions[region].append(gift)
    
    ## Done with the old trip Ids, remove in order to avoid confusion with the new ones
    del gifts_df['TripId']
    
    n_gifts_covered = 0
    dfs = []
    
    ## Put partition masks in the dataframe
    for i,p in enumerate(partitions):
        df['p{0}'.format(i)] = False
        
        for g in p:
            gi = df[df['GiftId'] == g].index.get_values()[0]
            df['p{0}'.format(i)][gi] = True

        p_df = df[df['p{0}'.format(i)]]
        
        ## Make the trips
        p_df = make_trips(p_df, maxd=maxd)
        p_df['TripId'] = p_df['TripId'] + n_gifts_covered
        n_gifts_covered += p_df['TripId'].max()
        
        dfs.append(p_df)
        
    return pd.concat(dfs)

In [45]:
gifts_df = pd.read_csv('gifts.csv')

In [46]:
gifts_df = make_trips(gifts_df)

In [47]:
gifts_df = partition(gifts_df, gap=31, maxd=1.0)

In [49]:
## Evaluate final score
x,n = score(gifts_df)
print 'Final score:', x, 'in', n, 'trips'

## To beat:  12541123905.8

Final score: 12558592884.5 in 2874 trips


In [ ]:
from cv2 import imread, cvtColor, COLOR_BGR2RGB


## ----------------------------------------------
def read_image(path):
    """
    Read an image and convert to RGB using openCV
    """
    img = imread(path)
    return cvtColor(img, COLOR_BGR2RGB)

In [ ]:
## Make a plot of the trajectories
earth = read_image('earth.jpg')

In [ ]:
def transform(x,y):
    """
    transform the latitude/longitude coordinates to the coordinates of the image
    """
    
    new_x = (x + 180)*(earth.shape[1]/360.0)
    new_y = -(y - 90)*(earth.shape[0]/180.0)
    
    return new_x, new_y

In [ ]:
import random
from matplotlib import colors

c = colors.cnames.keys()

fig = plt.figure(frameon=False)
fig.set_size_inches(100,50)

ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)

ax.imshow(earth, aspect='normal')

n_trips = gifts_df['TripId'].max()
for i in range(1, n_trips+1):
    trip = gifts_df[gifts_df['TripId'] == i]
    
    x = trip['Longitude'].values
    y = trip['Latitude'].values
    
    x_, y_ = transform(x,y)
    ax.plot(x_,y_,c=random.choice(c))
        
fig.savefig('trips.png')

In [ ]:
trips_df = gifts_df[['GiftId', 'TripId']]
trips_df.to_csv('trips.csv', index=False)

In [ ]:
## Next I need to figure out an algorithm that exchanges gifts between trips until it finds exchanges that
## benefit the overall score

In [ ]:
import copy

## --------------------------------------------------
def optimize_pair(tripA, tripB):
    """
    Try all exchanges between trips, keep the exchanges that reduces the total work
    """
    
    total_work = trip_work(tripA) + trip_work(tripB)
    
    nA = len(tripA)
    nB = len(tripB)

    for jA, rowA in tripA.iterrows():
        for jB, rowB in tripB.iterrows():
                
            rA = copy.copy(tripA.loc[jA].values)
            rB = copy.copy(tripB.loc[jB].values)

            tripA.loc[jA] = rB
            tripB.loc[jB] = rA
            
            new_work = trip_work(tripA) + trip_work(tripB)
            wA = tripA['Weight'].sum()
            wB = tripB['Weight'].sum()
        
            ## If the new work is better, keep the exchange and change the total work to beat
            if new_work < total_work and (wA < 990) and (wB < 990):
                total_work = new_work
            ## If the new work isn't better, undo the exchange
            else:
                tripA.loc[jA] = rA
                tripB.loc[jB] = rB

In [ ]:
import time
improvement = 0

t = time.time()

for i in range(n_trips-1):
    iA = i+1
    if i==0: iA == n_trips
    iB = i+2
    
    A = gifts_df[gifts_df['TripId'] == iA]
    B = gifts_df[gifts_df['TripId'] == iB]
    
    before = trip_work(A) + trip_work(B)
    
    optimize_pair(A,B)
    
    after = trip_work(A) + trip_work(B)
    
    improvement += before-after
    
    A['TripId'] = iA
    B['TripId'] = iB
    
    gifts_df[gifts_df['TripId'] == iA] = A
    gifts_df[gifts_df['TripId'] == iB] = B
    
    current_time = time.time()
    print i+1, 'Cumulative improvement:', improvement, 'dt:', current_time - t
    t = time.time()
    